In [1]:
!pip install networkx

In [ ]:
!pip install urllib3==1.26.3 boto3==1.26.163

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
!cp "/content/drive/MyDrive/Colab Notebooks (1)/smartJen/KG/PE4-6.xlsx" "PE.xlsx"

In [8]:
!cp "/content/drive/MyDrive/Colab Notebooks (1)/KG/dataset/Topic_relationship_Primary Maths.xlsx" "PM.xlsx"

File excel will be in the format as:


---


|---|---|---|---|---|

| head_id |	head	| rel	| tail_id	| tail|

| 75	    |  Modal Verb |	require	| 162 | 	Verbs of Being/Linking Verbs |




# **Function for DB connection**

In [63]:
import requests
import json
import pandas as pd
url_query = "https://crrxojhck2eaaahavn5wl7usri0nxjmb.lambda-url.ap-southeast-1.on.aws/dba/query"
def call_db(db_name, query):
    response = None
    try:
        response = requests.get(url_query, params={"db":db_name, "query":query}, headers={'Accept': 'application/json'})
        output = json.loads(response.text)
        print(output)
        return pd.DataFrame(output['body'])
    except Exception as e:
        print(response.text)
        print(e)

In [36]:
import os
import datetime
import logging
logger = logging.getLogger()
import requests
import boto3
import json
from botocore import UNSIGNED
from botocore.client import Config as AWSConfig

URL_QUERY = "https://spld938daj.execute-api.ap-southeast-1.amazonaws.com/recsys-db-query"
DB = "rnd"
BUCKER_NAME = "pdf-digitalization"


def request2s3(file_name, file_path, s3name=BUCKER_NAME):
        try:
            file_name = f"recsys/{file_name}"
            s3 = boto3.client('s3', config=AWSConfig(signature_version=UNSIGNED))
            s3.upload_file(file_path, s3name, file_name) #upload to s3.
            text = "https://{}.s3.amazonaws.com/{}".format(s3name, file_name)
            return text
        except Exception as e:
            print("S3 error", e)
            return None




# **Convert KG from excel files**

In [70]:
import pandas as pd
import networkx as nx

In [71]:
#@title read excel file : if you have multifiles, read each file then concatnating them
# df1=pd.read_excel("/content/math.xlsx")
# df2=pd.read_excel("/content/PE1.xlsx", header=None).rename(columns={0:'head', 1:'rel', 2:'tail'})
df2=pd.read_excel("/content/PE.xlsx")
# df = pd.concat([df1,df2], axis=0)
df = df2
df

,head_id,head,rel,tail_id,tail
0,75,Modal Verb (can/could/must/should/will/would/m...,require,162,Verbs of Being/Linking Verbs (be/is/am/are/was...
1,75,Modal Verb (can/could/must/should/will/would/m...,require,188,"Helping/Auxiliary verbs (did,have,had,been,nee..."
2,75,Modal Verb (can/could/must/should/will/would/m...,require,191,Verbs (action verbs)
3,75,Modal Verb (can/could/must/should/will/would/m...,is required by,201,"Mixed Tenses (Present, Past and Future)"
4,75,Modal Verb (can/could/must/should/will/would/m...,is required by,205,"Interrogative Pronouns (whose, what, which)"
...,...,...,...,...,...
3220,1434,Future Perfect Continuous,relevant,1436,Reading Aloud
3221,1434,Future Perfect Continuous,is required by,1437,Listening Comprehension
3222,1435,Stimulus Based Conversation,relevant,1436,Reading Aloud
3223,1435,Stimulus Based Conversation,relevant,1437,Listening Comprehension


In [72]:
#@title revert the relationship to make 2-direction graph
df_cp = df.copy()
df_cp['head_id'] = df['tail_id']
df_cp['tail_id'] = df['head_id']
df_cp['head'] = df['tail']
df_cp['tail'] = df['head']
df_cp['rel'] = df['rel'].apply(lambda x: "reverse of require" if x.strip().lower()=='require' else 'require' if x.strip().lower() == 'is required by' else 'relevant' if x.strip().lower() == 'relevant' else "")
dfmerge = pd.concat([df_cp, df])
dfmerge

,head_id,head,rel,tail_id,tail
0,162,Verbs of Being/Linking Verbs (be/is/am/are/was...,reverse of require,75,Modal Verb (can/could/must/should/will/would/m...
1,188,"Helping/Auxiliary verbs (did,have,had,been,nee...",reverse of require,75,Modal Verb (can/could/must/should/will/would/m...
2,191,Verbs (action verbs),reverse of require,75,Modal Verb (can/could/must/should/will/would/m...
3,201,"Mixed Tenses (Present, Past and Future)",require,75,Modal Verb (can/could/must/should/will/would/m...
4,205,"Interrogative Pronouns (whose, what, which)",require,75,Modal Verb (can/could/must/should/will/would/m...
...,...,...,...,...,...
3220,1434,Future Perfect Continuous,relevant,1436,Reading Aloud
3221,1434,Future Perfect Continuous,is required by,1437,Listening Comprehension
3222,1435,Stimulus Based Conversation,relevant,1436,Reading Aloud
3223,1435,Stimulus Based Conversation,relevant,1437,Listening Comprehension


In [73]:
dfmerge['check'] = dfmerge['rel'].apply(lambda x : x in ['require', 'relevant', 'reverse of require'])
dfmerge = dfmerge[dfmerge['check']][['head', 'rel', 'tail', 'head_id', 'tail_id']]
dfmerge.head(2)

,head,rel,tail,head_id,tail_id
0,Verbs of Being/Linking Verbs (be/is/am/are/was...,reverse of require,Modal Verb (can/could/must/should/will/would/m...,162,75
1,"Helping/Auxiliary verbs (did,have,had,been,nee...",reverse of require,Modal Verb (can/could/must/should/will/would/m...,188,75


In [74]:
#convert relationship "relevant-require" to 'revese_of_require - require'
extra_dfmerge = pd.DataFrame()
for i in range(len(dfmerge)):
    _, rel, _, head_id, tail_id = dfmerge.iloc[i]
    indices = dfmerge[(dfmerge['head_id'] == tail_id) & (dfmerge['tail_id'] == head_id)].index
    if len(indices)==0 or int(indices.values) >= len(dfmerge) : continue
    rel_ = dfmerge.iloc[int(indices[0]), 1]
    if rel == 'require':
        if rel_ == "relevant":
            dfmerge.iloc[int(indices.values), 1] = "reverse of require"
        elif rel_ == 'require':
            tmp = dfmerge.iloc[[i]]
            tmp['rel'] = 'reverse of require'
            extra_dfmerge = pd.concat([extra_dfmerge, tmp])
    elif rel == 'relevant' and rel_ == "require":
            dfmerge.iloc[i, 1] = "reverse of require"
dfmerge = pd.concat([dfmerge, extra_dfmerge])
dfmerge.reset_index(drop=True, inplace=True)


<ipython-input-74-1a9ae619e291>:6: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  if len(indices)==0 or int(indices.values) >= len(dfmerge) : continue
<ipython-input-74-1a9ae619e291>:10: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  dfmerge.iloc[int(indices.values), 1] = "reverse of require"
<ipython-input-74-1a9ae619e291>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp['rel'] = 'reverse of req

In [75]:
dfmerge

,head,rel,tail,head_id,tail_id
0,Verbs of Being/Linking Verbs (be/is/am/are/was...,reverse of require,Modal Verb (can/could/must/should/will/would/m...,162,75
1,"Helping/Auxiliary verbs (did,have,had,been,nee...",reverse of require,Modal Verb (can/could/must/should/will/would/m...,188,75
2,Verbs (action verbs),reverse of require,Modal Verb (can/could/must/should/will/would/m...,191,75
3,"Mixed Tenses (Present, Past and Future)",require,Modal Verb (can/could/must/should/will/would/m...,201,75
4,"Interrogative Pronouns (whose, what, which)",require,Modal Verb (can/could/must/should/will/would/m...,205,75
...,...,...,...,...,...
4955,Word Order,reverse of require,Past Perfect Continuous,397,1430
4956,Word Order,reverse of require,Future Simple,397,1431
4957,Word Order,reverse of require,Future Continuous (Progressive),397,1432
4958,Word Order,reverse of require,Future Perfect,397,1433


**Calculating weight of relation edge**


*   require = 2
*   relevant = 1
*   extra_weight = number of shared question / number question of topic

In [76]:
subject_id = 1
db_id = 1
######
# weight relationship edges
dfmerge['rel_weight'] = dfmerge['rel'].apply(lambda x: 1 if x=='relevant' else 2)

#calculate extra weight
query = f"""
    SELECT topic_id, sum(num) as num FROM (
        SELECT topic_id , count(*) as num FROM pals_questions WHERE topic_id > 0 AND subject_type={subject_id} and pals_dbid={db_id} GROUP BY topic_id
        UNION ALL
        SELECT topic_id2, count(*) as num FROM pals_questions WHERE topic_id2 > 0 AND subject_type={subject_id} and pals_dbid={db_id} GROUP BY topic_id2
        UNION ALL
        SELECT topic_id3, count(*) as num FROM pals_questions WHERE topic_id3 > 0 AND subject_type={subject_id} and pals_dbid={db_id} GROUP BY topic_id3
        UNION ALL
        SELECT topic_id4, count(*) as num FROM pals_questions WHERE topic_id4 > 0 AND subject_type={subject_id} and pals_dbid={db_id} GROUP BY topic_id4
    ) as t
    GROUP BY topic_id
    """
print(query)
num_question_in_topic = call_db('pals',query)
print(num_question_in_topic)
num_question_in_topic.set_index(['topic_id'], inplace=True)
query = f"""
    SELECT t1, t2, sum(num) as num FROM (
        SELECT topic_id as t1, topic_id2 as t2, count(*) as num FROM pals_questions WHERE topic_id > 0 AND topic_id2 > 0 AND subject_type={subject_id} and pals_dbid={db_id} GROUP BY topic_id, topic_id2
        UNION ALL
        SELECT topic_id, topic_id3, count(*) as num FROM pals_questions WHERE topic_id > 0 AND topic_id3 > 0 AND subject_type={subject_id} and pals_dbid={db_id} GROUP BY topic_id, topic_id3
        UNION ALL
        SELECT topic_id, topic_id4, count(*) as num FROM pals_questions WHERE topic_id > 0 AND topic_id4 > 0 AND subject_type={subject_id} and pals_dbid={db_id} GROUP BY topic_id, topic_id4

        UNION ALL
        SELECT topic_id2, topic_id,  count(*) as num FROM pals_questions WHERE topic_id > 0 AND topic_id2 > 0 AND subject_type={subject_id} and pals_dbid={db_id} GROUP BY topic_id2, topic_id
        UNION ALL
        SELECT topic_id2, topic_id3, count(*) as num FROM pals_questions WHERE topic_id2 > 0 AND topic_id3 > 0 AND subject_type={subject_id} and pals_dbid={db_id} GROUP BY topic_id2, topic_id3
        UNION ALL
        SELECT topic_id2, topic_id4, count(*) as num FROM pals_questions WHERE topic_id2 > 0 AND topic_id4 > 0 AND subject_type={subject_id} and pals_dbid={db_id} GROUP BY topic_id2, topic_id4

        UNION ALL
        SELECT topic_id3, topic_id,  count(*) as num FROM pals_questions WHERE topic_id > 0 AND topic_id3 > 0 AND subject_type={subject_id} and pals_dbid={db_id} GROUP BY topic_id3, topic_id
        UNION ALL
        SELECT topic_id3, topic_id2, count(*) as num FROM pals_questions WHERE topic_id2 > 0 AND topic_id3 > 0 AND subject_type={subject_id} and pals_dbid={db_id} GROUP BY topic_id3, topic_id2
        UNION ALL
        SELECT topic_id3, topic_id4, count(*) as num FROM pals_questions WHERE topic_id3 > 0 AND topic_id4 > 0 AND subject_type={subject_id} and pals_dbid={db_id} GROUP BY topic_id3, topic_id4

        UNION ALL
        SELECT topic_id4, topic_id,  count(*) as num FROM pals_questions WHERE topic_id > 0 AND topic_id4 > 0 AND subject_type={subject_id} and pals_dbid={db_id} GROUP BY topic_id4, topic_id
        UNION ALL
        SELECT topic_id4, topic_id2, count(*) as num FROM pals_questions WHERE topic_id2 > 0 AND topic_id4 > 0 AND subject_type={subject_id} and pals_dbid={db_id} GROUP BY topic_id4, topic_id2
        UNION ALL
        SELECT topic_id4, topic_id3, count(*) as num FROM pals_questions WHERE topic_id3 > 0 AND topic_id4 > 0 AND subject_type={subject_id} and pals_dbid={db_id} GROUP BY topic_id4, topic_id3
    ) as t
    GROUP BY t1, t2
    """
print(query)
topic_pair = call_db('pals',query)


topic_pair




    SELECT topic_id, sum(num) as num FROM (
        SELECT topic_id , count(*) as num FROM pals_questions WHERE topic_id > 0 AND subject_type=1 and pals_dbid=1 GROUP BY topic_id
        UNION ALL
        SELECT topic_id2, count(*) as num FROM pals_questions WHERE topic_id2 > 0 AND subject_type=1 and pals_dbid=1 GROUP BY topic_id2
        UNION ALL
        SELECT topic_id3, count(*) as num FROM pals_questions WHERE topic_id3 > 0 AND subject_type=1 and pals_dbid=1 GROUP BY topic_id3
        UNION ALL
        SELECT topic_id4, count(*) as num FROM pals_questions WHERE topic_id4 > 0 AND subject_type=1 and pals_dbid=1 GROUP BY topic_id4
    ) as t
    GROUP BY topic_id
    
{'complete': True, 'body': {'topic_id': {'0': 313, '1': 319, '2': 380, '3': 278, '4': 304, '5': 305, '6': 1423, '7': 386, '8': 379, '9': 334, '10': 251, '11': 335, '12': 327, '13': 333, '14': 371, '15': 369, '16': 322, '17': 385, '18': 1424, '19': 1427, '20': 325, '21': 328, '22': 75, '23': 205, '24': 384, '25': 372, '2

,t1,t2,num
0,1423,162,28.0
1,379,1427,161.0
2,335,379,92.0
3,379,1429,20.0
4,333,332,95.0
...,...,...,...
735,1425,346,1.0
736,322,372,1.0
737,380,323,1.0
738,380,370,2.0


## add info of substrand to graph

In [31]:
query = f"""
    SELECT id as topic_id, substrand_id FROM pals_topics where subject_type={subject_id} and pals_dbid={db_id}
    """
print(query)
topic_to_substrand = call_db('pals',query)
print(topic_to_substrand )
topic_to_substrand = topic_to_substrand.set_index(['topic_id'])['substrand_id'].to_dict()



    SELECT id as topic_id, substrand_id FROM pals_topics where subject_type=1 and pals_dbid=1
    
{'complete': True, 'body': {'topic_id': {'0': 75, '1': 188, '2': 191, '3': 201, '4': 205, '5': 162, '6': 225, '7': 235, '8': 245, '9': 304, '10': 305, '11': 251, '12': 313, '13': 317, '14': 319, '15': 320, '16': 321, '17': 322, '18': 323, '19': 324, '20': 325, '21': 326, '22': 327, '23': 328, '24': 332, '25': 333, '26': 334, '27': 335, '28': 336, '29': 337, '30': 338, '31': 339, '32': 340, '33': 341, '34': 342, '35': 343, '36': 344, '37': 345, '38': 346, '39': 347, '40': 348, '41': 353, '42': 354, '43': 278, '44': 356, '45': 369, '46': 370, '47': 371, '48': 372, '49': 373, '50': 374, '51': 376, '52': 379, '53': 380, '54': 382, '55': 383, '56': 384, '57': 385, '58': 386, '59': 387, '60': 391, '61': 394, '62': 395, '63': 396, '64': 397, '65': 414, '66': 1434, '67': 1435, '68': 1436, '69': 1437, '70': 1423, '71': 1424, '72': 1425, '73': 1426, '74': 1427, '75': 1428, '76': 1429, '77': 1430, 

In [77]:
for _, t1, t2, shared_num in topic_pair.itertuples():
    weight = shared_num / (num_question_in_topic.loc[t1, 'num'] + 1e-5)
    head_id, tail_id = t1, t2
    indices = dfmerge[(dfmerge['head_id'] == head_id) & (dfmerge['tail_id'] == tail_id)].index
    for i in indices:
        dfmerge.iloc[i, -1] += round(weight, 3)

In [78]:
dfmerge['head_substrand'] = dfmerge['head_id'].apply(lambda x: topic_to_substrand[int(x)] if x in topic_to_substrand else None)
dfmerge['tail_substrand'] = dfmerge['tail_id'].apply(lambda x: topic_to_substrand[int(x)] if x in topic_to_substrand else None)
dfmerge.head(4)

,head,rel,tail,head_id,tail_id,rel_weight,head_substrand,tail_substrand
0,Verbs of Being/Linking Verbs (be/is/am/are/was...,reverse of require,Modal Verb (can/could/must/should/will/would/m...,162,75,2.000,86,86
1,"Helping/Auxiliary verbs (did,have,had,been,nee...",reverse of require,Modal Verb (can/could/must/should/will/would/m...,188,75,2.028,86,86
2,Verbs (action verbs),reverse of require,Modal Verb (can/could/must/should/will/would/m...,191,75,2.000,86,86
3,"Mixed Tenses (Present, Past and Future)",require,Modal Verb (can/could/must/should/will/would/m...,201,75,2.000,85,86


In [79]:
for i, row in dfmerge.iterrows():
  print(i)
  print(row)
  break

0
head              Verbs of Being/Linking Verbs (be/is/am/are/was...
rel                                              reverse of require
tail              Modal Verb (can/could/must/should/will/would/m...
head_id                                                         162
tail_id                                                          75
rel_weight                                                      2.0
head_substrand                                                   86
tail_substrand                                                   86
Name: 0, dtype: object


In [81]:
#@title seperate df to level --> should rewrite in future
new_edges = []
for lvl in [1, 2, 3, 4, 5, 6]:
  for i, row in dfmerge.iterrows():
    u = row.head_id
    v = row.tail_id
    head_substrand = row["head_substrand"]
    tail_substrand = row["tail_substrand"]
    rel_weight = row['rel_weight']
    new_edge = {
        'head_id':f't-{u}-l{lvl}',
        'tail_id':f't-{v}-l{lvl}',
         'rel'  : row["rel"],
         'rel_weight'  : rel_weight,
         'head_substrand': head_substrand,
         'tail_substrand':tail_substrand

    }
    new_edges += [new_edge]
len(new_edges)

29760

In [82]:
dfmerge = pd.DataFrame(new_edges)
dfmerge.head()

,head_id,tail_id,rel,rel_weight,head_substrand,tail_substrand
0,t-162-l1,t-75-l1,reverse of require,2.000,86,86
1,t-188-l1,t-75-l1,reverse of require,2.028,86,86
2,t-191-l1,t-75-l1,reverse of require,2.000,86,86
3,t-201-l1,t-75-l1,require,2.000,85,86
4,t-205-l1,t-75-l1,require,2.000,83,86


In [83]:
topic_graph = nx.from_pandas_edgelist(dfmerge, 'head_id', 'tail_id',
                    edge_attr=["rel", 'rel_weight', 'head_substrand',  'tail_substrand'],
                    create_using=nx.MultiDiGraph())

In [84]:
# save graph and push into S3
import os
db_id=1
def submit_to_link_table(file_path, model_name, category='KG', db_id=1):
    try:
        file_name = os.path.basename(file_path)
        extention = file_name.split(".")[-1]
        link = request2s3(file_name=file_name, file_path=file_path )
        updated =  datetime.datetime.now().strftime("%y-%m-%d %H:%M:%S")
        if link is None: raise Exception("ERROR in uploading to S3")
        call_db('pals', f"INSERT INTO  pals_model_config_link (db_id, link, category, model_name, updated) VALUES ({db_id},'{link}', '{category}', '{model_name}', '{updated}')")
        call_db('pals', f"DELETE  FROM  pals_model_config_link WHERE db_id={db_id}, category='{category}' and model_name='{model_name}' and  updated < '{updated}'")
    except Exception as e:
        logger.exception(str(e))
        print(f"Cannot upload file-->{file_path} because of {str(e)}")

tmproot=""
db_id=1
subject_id=1
save_name = f"bigKG_subject_{subject_id}_dbid_{db_id}"
model_name = f"bigKG_subject_{subject_id}"
nx.write_gml(topic_graph, os.path.join(tmproot, save_name+'.gml'))
submit_to_link_table(os.path.join(tmproot, save_name+'.gml'), model_name, 'KG', db_id)

{'complete': True, 'body': {}}
{'complete': False, 'body': None}


In [85]:
call_db('pals', f"SELECT *  FROM  pals_model_config_link")

{'complete': True, 'body': {'id': {'0': 1, '1': 2, '2': 32, '3': 33, '4': 34, '5': 35, '6': 36, '7': 37, '8': 38, '9': 39, '10': 40, '11': 41, '12': 46, '13': 47}, 'db_id': {'0': 1, '1': 1, '2': 1, '3': 1, '4': 1, '5': 1, '6': 1, '7': 1, '8': 1, '9': 1, '10': 1, '11': 1, '12': 1, '13': 1}, 'link': {'0': 'https://pdf-digitalization.s3.amazonaws.com/recsys/topicG_subject_2.gml', '1': 'https://pdf-digitalization.s3.amazonaws.com/recsys/topicG_subject_1.gml', '2': 'https://pdf-digitalization.s3.amazonaws.com/model/dbid1-subject_1_100_ktmodel.json', '3': 'https://pdf-digitalization.s3.amazonaws.com/model/dbid1-subject_1_100_ktmodel.pth', '4': 'https://pdf-digitalization.s3.amazonaws.com/model/dbid1-subject_1_120_ktmodel.json', '5': 'https://pdf-digitalization.s3.amazonaws.com/model/dbid1-subject_1_120_ktmodel.pth', '6': 'https://pdf-digitalization.s3.amazonaws.com/model/dbid1-subject_2_120_ktmodel.json', '7': 'https://pdf-digitalization.s3.amazonaws.com/model/dbid1-subject_2_120_ktmodel.pth

,id,db_id,link,category,model_name,updated,version
0,1,1,https://pdf-digitalization.s3.amazonaws.com/re...,topic-graph,topicG_subject_2,2024-04-14 08:11:55,None
1,2,1,https://pdf-digitalization.s3.amazonaws.com/re...,topic-graph,topicG_subject_1,2024-04-14 08:11:55,None
2,32,1,https://pdf-digitalization.s3.amazonaws.com/mo...,kt-model,subject_1_100_ktmodel.json,2024-04-16 13:15:17,None
3,33,1,https://pdf-digitalization.s3.amazonaws.com/mo...,kt-model,subject_1_100_ktmodel.pth,2024-04-16 13:15:17,None
4,34,1,https://pdf-digitalization.s3.amazonaws.com/mo...,kt-model,subject_1_120_ktmodel.json,2024-04-16 13:15:17,None
5,35,1,https://pdf-digitalization.s3.amazonaws.com/mo...,kt-model,subject_1_120_ktmodel.pth,2024-04-16 13:15:17,None
6,36,1,https://pdf-digitalization.s3.amazonaws.com/mo...,kt-model,subject_2_120_ktmodel.json,2024-04-16 13:15:17,None
7,37,1,https://pdf-digitalization.s3.amazonaws.com/mo...,kt-model,subject_2_120_ktmodel.pth,2024-04-16 13:15:17,None
8,38,1,https://pdf-digitalization.s3.amazonaws.com/mo...,kt-model,subject_3_120_ktmodel.json,2024-04-16 13:15:17,None
9,39,1,https://pdf-digitalization.s3.amazonaws.com/mo...,kt-model,subject_3_120_ktmodel.pth,2024-04-16 13:15:17,None
